# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!


  1. Use RAGAS to Generate Synthetic Data
  2. Load them into a LangSmith Dataset
  3. Evaluate our RAG chain against the synthetic test data
  4. Make changes to our pipeline
  5. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMINN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ADMINN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

LangChain API Key: ········


We'll also want to set a project name to make things easier for ourselves.

In [6]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"PSI - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [7]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [8]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "bills/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [10]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [11]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [12]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node 58038132-80b2-472b-b0bc-0b29337d72d5 does not have a summary. Skipping filtering.
Node 083f6085-a681-4aaf-98d9-7f2e91002272 does not have a summary. Skipping filtering.
Node ae2945a2-1dcd-4535-8f13-5ddcb1bcff38 does not have a summary. Skipping filtering.
Node 50934c92-9cd8-4516-b2e0-d764e08196a9 does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 20, relationships: 136)

We can save and load our knowledge graphs as follows.

In [13]:
kg.save("bills/ai_law.json")
bills_data_kg = KnowledgeGraph.load("bills/ai_law.json")
bills_data_kg

KnowledgeGraph(nodes: 20, relationships: 136)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=bills_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [15]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

</div>


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

* The three types of query synthesizers generate different kinds of questions for evaluating retrieval-augmented generation systems: the SingleHopSpecificQuerySynthesizer creates simple, direct questions answerable with a single fact; the MultiHopAbstractQuerySynthesizer generates more complex, conceptual questions that require connecting multiple pieces of information; and the MultiHopSpecificQuerySynthesizer produces detailed, focused questions that also require combining several facts, but in a concrete and specific way.
</span>


Finally, we can use our `TestSetGenerator` to generate our testset!

In [16]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What role does the European Union play in AI r...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The European Union's landmark AI Act of 2024 h...,single_hop_specifc_query_synthesizer
1,What Georgetown University do about AI risks?,[AI presents enormous opportunities for the Ph...,The context does not specify actions taken by ...,single_hop_specifc_query_synthesizer
2,Can you explain what AIRA stands for and its s...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,AIRA refers to the Artificial Intelligence Reg...,single_hop_specifc_query_synthesizer
3,How AI is promote innovation and responsible u...,"[1 \na) Promote innovation, technological adva...",The context states that AI should promote inno...,single_hop_specifc_query_synthesizer
4,What does AGI mean in AI and why is it importa...,[1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11\n12\n13\n1...,Artificial General Intelligence (AGI) refers t...,single_hop_specifc_query_synthesizer
5,"How does the review process of AI risks, inclu...","[<1-hop>\n\n1 \na) Promote innovation, technol...",The review process of AI risks involves the AI...,multi_hop_abstract_query_synthesizer
6,How does the naic promote AI in sectors like a...,[<1-hop>\n\n1 \nSec. 8. NAICSecretariat. - The...,The NAIC promotes AI in sectors such as agricu...,multi_hop_abstract_query_synthesizer
7,H0w do penalties for AI-related crimminal acti...,[<1-hop>\n\n1 \ndevelopment priorities. These ...,The context indicates that penalties for AI-re...,multi_hop_abstract_query_synthesizer
8,how ASI surpass humans and AI risks like hallu...,[<1-hop>\n\n1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11...,the context explains that Artificial Superinte...,multi_hop_specific_query_synthesizer
9,How does the DICT's role in AI regulation rela...,[<1-hop>\n\n1 \nSec. 6. Jurisdiction of the NA...,"The DICT is part of the NAIC, which has jurisd...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [17]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node eff3d17f-91d5-42a9-afc9-072104dd2935 does not have a summary. Skipping filtering.
Node 7192e678-e847-4cc1-bf66-b02ec953b514 does not have a summary. Skipping filtering.
Node f3b73e2b-ccfb-411e-b001-3a78ac8463e8 does not have a summary. Skipping filtering.
Node f4d48441-df79-4be1-9f9f-969d78e64e44 does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [18]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the role of CD0 Magazine in discussing...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,"According to the provided context, CD0 Magazin...",single_hop_specifc_query_synthesizer
1,What are the potential risks associated with t...,[AI presents enormous opportunities for the Ph...,The context indicates that Artificial Superint...,single_hop_specifc_query_synthesizer
2,What is Article XIV?,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,Article XIV is a section of the Philippine Con...,single_hop_specifc_query_synthesizer
3,What is the significance of ASI in the context...,"[1 \na) Promote innovation, technological adva...",The context states that the Act shall regulate...,single_hop_specifc_query_synthesizer
4,How do intrinsic hallucination and extrinsic h...,[<1-hop>\n\n1 \nground its responses in verifi...,Intrinsic hallucination occurs when the AI mod...,multi_hop_abstract_query_synthesizer
5,How does AI regulation address worker displace...,[<1-hop>\n\n1 \nd) Take full responsibility fo...,The AI regulation framework requires employers...,multi_hop_abstract_query_synthesizer
6,support for tech vocational education and disc...,[<1-hop>\n\n1 \niii) \nProof of employer engag...,The context shows that there is support for te...,multi_hop_abstract_query_synthesizer
7,what are the responsibilities of AI developers...,[<1-hop>\n\n1 \nSec. 15. AI Ethics Review Boar...,"The responsibilities of AI developers, deploye...",multi_hop_abstract_query_synthesizer
8,How does TESDA contribute to AI regulation and...,[<1-hop>\n\n1 \nd) Take full responsibility fo...,TESDA is involved in AI regulation and workfor...,multi_hop_specific_query_synthesizer
9,How does the NAIC's regulation of AI in employ...,[<1-hop>\n\n1 \nd) Take full responsibility fo...,The NAIC's regulation of AI in employment and ...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [20]:
from langsmith import Client

client = Client()

dataset_name = "Philippines AI Bills"

try:
    # Try to get the existing dataset
    langsmith_dataset = client.read_dataset(dataset_name=dataset_name)
    print(f"Using existing dataset: {dataset_name}")
except Exception:
    # If it doesn't exist, create it
    langsmith_dataset = client.create_dataset(
        dataset_name=dataset_name,
        description="Philippines AI Bills"
    )
    print(f"Created new dataset: {dataset_name}")

Using existing dataset: Philippines AI Bills


We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [21]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [22]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [24]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [25]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG"
)

In [26]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [27]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [29]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [30]:
rag_chain.invoke({"question" : "How much is the penalty for spreading disinformation?"})

'The penalty for using AI to create or disseminate disinformation is:\n\n- A fine of One Million Pesos (Php 1,000,000) to Five Million Pesos (Php 5,000,000),  \n- or imprisonment of three (3) years to ten (10) years,  \n- or both, at the discretion of the court.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [31]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [32]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

* qa_evaluator: This evaluator checks if the model’s answer is correct and relevant to the question, using an LLM to compare the generated answer with the reference answer.

* labeled_helpfulness_evaluator: This evaluator measures how helpful the model’s answer is to the user, specifically considering if the answer would assist the user when compared to the correct reference answer. It uses an LLM and a custom “helpfulness” criterion.
</div>

## LangSmith Evaluation

In [33]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'mealy-shape-82' at:
https://smith.langchain.com/o/7fc042b4-97e0-5f3f-b110-30a87bc46433/datasets/2e682626-7edb-4606-ad73-4627a7519185/compare?selectedSessions=35f267b3-9e14-4f42-a0f3-425327fe27b0




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do the penalties imposed by the DOST for A...,I don't know.,None,The penalties imposed by the DOST for AI-relat...,0,0,0,1.108801,9dce146f-36f2-463f-b8b9-badab90cee69,b0fe8095-82b7-4c10-9245-384acfff2f47
1,How does the DOST contribute to the developmen...,"Based on the provided context, the Department ...",None,The DOST contributes to AI-related employment ...,0,0,0,5.096888,bf43c5bd-6c53-4285-b11c-e47e5a88f21f,4c866ff5-d2d3-488e-87fc-63145b7cce08
2,How does the NAIC's regulation of AI in employ...,"Based on the provided context, the NAIC is res...",None,The NAIC's regulation of AI in employment and ...,0,0,0,5.471050,570050a8-5f4d-4cbd-b8c1-c8e5521bb151,26cdaf5d-e519-431d-a729-ca8ae7b31459
3,How does TESDA contribute to AI regulation and...,"According to the context, TESDA contributes to...",None,TESDA is involved in AI regulation and workfor...,1,0,0,2.202665,f62dffa5-225f-4663-bd7f-8aa3c52784c3,75e20600-3f61-4b9d-b05c-2d64d6622c8f
4,what are the responsibilities of AI developers...,"Based on the provided context, AI developers, ...",None,"The responsibilities of AI developers, deploye...",1,0,0,3.341976,2adb0549-7c0b-4d29-bc70-0a569e96ec66,cec7c75e-d1f8-467c-9b81-afce3c113f9f
5,support for tech vocational education and disc...,"Based on the provided context, the Act promote...",None,The context shows that there is support for te...,0,0,0,5.491835,c8370608-4d85-4e65-a43e-3c3ce8a55a68,fd57ffd0-3bfc-4406-9715-88fe0fc6ed8e
6,How does AI regulation address worker displace...,AI regulation addresses worker displacement an...,None,The AI regulation framework requires employers...,1,1,0,10.487035,f273ef67-9a8f-4ad9-9b9e-4bd652854ff8,c15a4f1d-299a-46bf-9d31-cb21cb044b2b
7,How do intrinsic hallucination and extrinsic h...,Intrinsic hallucination occurs when an AI mode...,None,Intrinsic hallucination occurs when the AI mod...,1,1,0,3.298744,d40085c2-0202-4e88-892c-a6c40bc697f6,af88a69b-0832-4e18-a061-a36b0bc182aa
8,What is the significance of ASI in the context...,The significance of Artificial Superintelligen...,None,The context states that the Act shall regulate...,1,1,0,3.475439,0bc32b5f-db58-4757-9f90-9b9314daf362,6c8de114-9ab4-46e8-9171-8539cf5ed241
9,What is Article XIV?,I don't know.,None,Article XIV is a section of the Philippine Con...,0,0,0,1.011127,7286e944-6353-464d-a512-07460b952588,f4a344f3-6691-4b54-a5f9-b52d02a027d3


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [34]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [35]:
rag_documents = docs

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer: 

Modifying the chunk size changes how much text is grouped together and retrieved at once from the documents. If the chunks are too small, important context might be split up and missed by the model, leading to incomplete or less accurate answers. If the chunks are too large, irrelevant information may be included, making it harder for the model to find the most relevant details. Therefore, finding the right chunk size helps balance context and relevance, directly impacting the quality and performance of the application's responses.

</div>

In [37]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

Modifying the embedding model changes how the application represents and understands the meaning of text. Different embedding models vary in their ability to capture semantic relationships, context, and nuances in language. A more advanced or better-suited embedding model can improve how well the system matches questions to relevant information, leading to more accurate and helpful responses. While a less capable model might miss important connections, reducing performance. So, the choice of embedding model directly affects the quality of retrieval and, ultimately, the application's effectiveness.

</div>

In [38]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG 2"
)

In [39]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [40]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [41]:
empathy_rag_chain.invoke({"question" : "Why is the Philippines AI Bill important?"})

'Thank you for your thoughtful question. Based on the context provided, the Philippines AI Bill is important because it aims to carefully balance the encouragement of technological innovation with ensuring that AI systems remain safe, ethical, transparent, and under meaningful human oversight. It recognizes that AI is still in its early stages and provides a general framework to promote responsible and lawful AI development.\n\nMoreover, the bill envisions a future where AI supports Filipino ingenuity, tackles national development challenges, and protects the rights and welfare of every citizen. It also underscores the State’s responsibility to prevent AI from being used to commit crimes, violate rights, or cause harm, whether deliberately or accidentally, all while fostering technological progress.\n\nI appreciate your interest in understanding this significant legislation that seeks to guide AI innovation thoughtfully and responsibly in the Philippines. If you have more questions or 

Finally, we can evaluate the new chain on the same test set!

In [42]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'artistic-acknowledgment-32' at:
https://smith.langchain.com/o/7fc042b4-97e0-5f3f-b110-30a87bc46433/datasets/2e682626-7edb-4606-ad73-4627a7519185/compare?selectedSessions=b0594322-bfb8-49c0-acee-1a8c260627ee




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do the penalties imposed by the DOST for A...,Thank you for your thoughtful question. Based ...,None,The penalties imposed by the DOST for AI-relat...,1,1,1,4.133611,9dce146f-36f2-463f-b8b9-badab90cee69,1cf2a76d-997c-4ae6-abbc-5855413f3c89
1,How does the DOST contribute to the developmen...,Thank you for asking such a thoughtful questio...,None,The DOST contributes to AI-related employment ...,1,1,1,7.068426,bf43c5bd-6c53-4285-b11c-e47e5a88f21f,ad0f5828-f766-42f3-a27d-fffb4fbb196b
2,How does the NAIC's regulation of AI in employ...,Thank you for your thoughtful question. Based ...,None,The NAIC's regulation of AI in employment and ...,1,0,1,5.375062,570050a8-5f4d-4cbd-b8c1-c8e5521bb151,f1db498e-4cc3-4763-a9c3-ca61ee5f2ce6
3,How does TESDA contribute to AI regulation and...,Thank you for your thoughtful question. It’s c...,None,TESDA is involved in AI regulation and workfor...,1,0,1,5.420858,f62dffa5-225f-4663-bd7f-8aa3c52784c3,5a2f7502-48ca-4224-89f5-09559bee64dc
4,what are the responsibilities of AI developers...,Thank you for your thoughtful question about t...,None,"The responsibilities of AI developers, deploye...",1,0,1,6.644590,2adb0549-7c0b-4d29-bc70-0a569e96ec66,40132d8d-2fd3-4e87-ab3a-960be9fa1c17
5,support for tech vocational education and disc...,Thank you for your thoughtful question. Based ...,None,The context shows that there is support for te...,0,0,1,3.952384,c8370608-4d85-4e65-a43e-3c3ce8a55a68,998d06c3-6a73-42bd-89e9-0b86ca4b957e
6,How does AI regulation address worker displace...,Thank you for your thoughtful question. Based ...,None,The AI regulation framework requires employers...,1,1,1,11.256042,f273ef67-9a8f-4ad9-9b9e-4bd652854ff8,39092809-2caa-453a-b7dc-b0d7176ab356
7,How do intrinsic hallucination and extrinsic h...,Thank you for your thoughtful question. Based ...,None,Intrinsic hallucination occurs when the AI mod...,1,1,1,3.389153,d40085c2-0202-4e88-892c-a6c40bc697f6,d59233b9-1dd6-4487-a9af-f5a9d05212b7
8,What is the significance of ASI in the context...,Thank you for your thoughtful question. Based ...,None,The context states that the Act shall regulate...,1,1,1,5.493394,0bc32b5f-db58-4757-9f90-9b9314daf362,752e637a-28c2-42fd-babd-81cd91137dcd
9,What is Article XIV?,Thank you for your thoughtful question. Based ...,None,Article XIV is a section of the Philippine Con...,1,0,1,4.315485,7286e944-6353-464d-a512-07460b952588,4226e2e8-2193-48cf-b47b-3c435a916268


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #4:

Explain why you believe certain metrics changed in certain ways, and provide a screenshot of the difference between the two chains.

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

Changes in metrics occur because modifications to chunk size, embedding models, or prompt design directly impact how effectively the system retrieves and answers questions. For example, a better embedding model or optimal chunk size can improve faithfulness, answer relevancy, and correctness by enabling the system to find and use more relevant information. While, suboptimal settings may lower precision or recall, affecting these metrics negatively. Comparing screenshots of evaluation results before and after such changes visually highlights these metric differences, illustrating the impact of the adjustments on system performance.

</div>